# Инференс LLM (server-версия) с vLLM и балансировка нагрузки

В этом ноутбуке мы рассмотрим основные аспекты инференса больших языковых моделей (LLM) с помощью сервера [vLLM](https://github.com/vllm-project/vllm) в контейнерах Docker. Также затронем вопросы масштабирования и балансировки нагрузки. Затрнонем Tensor и Pipeline Parralelism. Рабочий пример инференса языковой модели c Docker можно найти в github [LLM inference project](https://github.com/ivan-digital/aquarius/tree/main/inference).

## 1. Серверный сценарий vs локальный Python
При **локальном** запуске нужно установить `vllm`, `transformers` и т.п. обычным `pip install`, а затем вызывать `LLM.generate(...)`. Но в **продакшен**-среде обычно:
- У вас есть **Docker-контейнер** или подобный механизм, где модель развёрнута в виде **серверного приложения**.
- Клиенты (web-приложение, другие сервисы) отправляют запросы на API vLLM (REST/HTTP или gRPC).
- В случае больших нагрузок мы создаём **несколько экземпляров** (реплик) сервиса и распределяем между ними запросы.

## 2. Что такое инференс?
Инференс (inference) — это процесс генерации ответа на основе обученной большой языковой модели, в нашем случае — модели `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B`. С точки зрения пользователя, это выглядит так:
1. Отправляем запрос с промптом (prompt) в сервис.
2. Модель (в backend) обрабатывает токены, обновляет кэш внимания (attention key-value) и начинает процесс декодирования (beam search, top-k, и т. д.).
3. Возвращается сгенерированный ответ.

## 3. Техники генерации
### 3.1 Beam Search
Beam Search — жадно-детерминированный алгоритм декодирования, в котором на каждом шаге поддерживается заданное число (ширина beam, k) наиболее вероятных гипотез последовательности. После генерации всех возможных продолжений из этих k веток выбираются снова k лучших.

Позволяет находить более вероятные целиком сформированные фрагменты текста по сравнению с жадным (greedy) поиском.
Даёт более стабильные и «правдоподобные» ответы, но при этом снижает разнообразие в сравнении с методами сэмплинга (особенно при высокой температуре).

### 3.2 Top-k / Top-p (Nucleus) Sampling
- **Top-k**: выбираем следующий токен из k наиболее вероятных.
- **Top-p**: выбираем токен из области, чья суммарная вероятность не превышает p.
Обе техники делают ответы более разнообразными.

### 3.3 Температура
Параметр температуры (T) — коэффициент масштабирования логитов перед применением softmax, влияющий на «остроту» распределения вероятностей при выборе следующего токена.

- При T = 1.0 преобразование логитов не меняет исходное распределение: модель выбирает токены строго пропорционально их вероятностям.
- При T < 1.0 (например, 0.7) распределение становится более «резким»: наиболее вероятные токены получают ещё больший вес, и ответы выходят более детерминированными.
- При T > 1.0 (например, 1.3) распределение «сглаживается»: вероятности менее частотных токенов растут, за счёт чего генерация становится более разнообразной и «креативной».



## 4. Оптимизации для LLM-инференса
### 4.1 Paged Attention
Paged Attention — это метод управления памятью при вычислении механизма внимания (attention) в трансформерах. Основная проблема, которую он решает, — это резкое возрастание потребления GPU-памяти при увеличении длины последовательности токенов. В стандартном механизме внимания память расходуется пропорционально O(n<sup>2</sup>), где n — длина входной последовательности. Для больших n объём выделенной памяти может быстро выйти за пределы доступного пула на видеокарте.

Paged Attention разбивает вычисления на «страницы» (pages) или блоки — подобно тому, как работает виртуальная память на уровне операционной системы.

### 4.2 FlashAttention / Kernel Fusion
FlashAttention — это высокоэффективная реализация механизма внимания, которая минимизирует число обращений к памяти и перестраивает логику вычислений таким образом, чтобы аккумулировать промежуточные результаты прямо в регистрах GPU и/или блоках быстрой кэш-памяти. При обычном подходе каждый элемент внимания требует многократных операций чтения/записи с/в глобальную память GPU, что довольно «дорого» по времени.

### 4.3 Квантование (int8, int4)
Квантование переводит веса модели из более высокой точности (FP32, FP16, BF16) в более низкую (например, int8, int4). Цель — уменьшить:

Объём памяти, занимаемый моделью (порой на 50–75% и более).
Пропускную способность, требуемую для чтения/записи весов.
Время вычислений (за счёт эффективных инструкций, например, INT8-множения на современных GPU/TPU).
При этом может слегка (или значительно, в зависимости от агрессивности квантования) ухудшиться точность модели.

#### Виды квантования

- Post-training quantization (PTQ): квантование после обучения, без дополнительного тренинга. Самый простой способ, но может сильнее повлиять на точность, так как веса меняются «резко» с FP16/FP32 до int8 и т.п.

- Quantization-aware training (QAT): модель обучается с учётом квантования, то есть «эмулирует» низкую точность во время обучения. Это обычно даёт более высокое качество результирующей модели, но требует дополнительного времени и ресурсов на тренировку.

- Mixed Precision: можно квантовать не все части модели, а только, например, некоторые слои или определённые типы операций (активации, веса и т. д.). Это помогает найти баланс между производительностью и качеством.

### 4.4 Speculative decoding
Генерация выполняется сразу двумя моделями: маленькая draft-model предлагает пачку токенов, а большая target-model проверяет и «отбрасывает» неподходящие, тем самым экономя вызовы тяжёлой модели и ускоряя вывод до ×2–×4 на тех же GPU. [vllm документация](https://docs.vllm.ai/en/latest/features/spec_decode.html) Задействуется через специальный параметр `speculative_config`.


## 5. Балансировка нагрузки (Load Balancing)
При **увеличении числа запросов** вы можете масштабировать сервис, поднимая **несколько** инстансов vLLM. Каждый контейнер будет использовать тот же образ (с той же моделью `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B`) — всё зависит от вашей GPU-инфраструктуры:
- Если у вас несколько GPU на одном сервере, можно поднять столько контейнеров, сколько GPU.
- Если есть несколько серверов, каждый с GPU, можно организовать кластер (docker swarm, kubernetes и т. д.).

### 5.1 Роль Nginx
Nginx (или другой веб-сервер/прокси) может выступать в роли **reverse proxy** и **load balancer**:
1. **Принимает** входящие запросы клиентов (запросы к REST API).
2. **Распределяет** эти запросы по нескольким контейнерам vLLM. Например, используя `upstream` в `nginx.conf`.
3. Возвращает клиенту ответ от одного из экземпляров.

Таким образом, когда нагрузка растёт, вы поднимаете больше реплик (например, `replicas: 2` или `replicas: 3` и т. д.), и Nginx "размазывает" входящий трафик по ним.

## 6. Параллелизм модели (Рассмотрим теоретически)
Более детальный материал [Model Parralelism](https://huggingface.co/docs/transformers/v4.13.0/parallelism) на сайте transformers. И туториал от vllm о том, [как выбирать распределенную архитектуру инференса языковых моделей](https://docs.vllm.ai/en/v0.5.2/serving/distributed_serving.html).

### 6.1 Tensor Parallelism (TP)

Огромные весовые матрицы делятся по строкам/столбцам; каждая GPU хранит и умножает лишь свою «долю» тензора .
vLLM-флаг. `--tensor-parallel-size <N>` — число GPU в узле, участвующих в TP.

Применяется эта техника когда модель не помещается в память одной GPU, но суммарная память узла достаточна; при наличии NVLink задержка минимальна.

### 6.2 Pipeline Parallelism (PP)

Полные слои трансформера распределяются по разным процессам/узлам; микробатчи проходят через «конвейер» из K стадий.
vLLM-флаг. `--pipeline-parallel-size <K>` — число стадий; на мульти-ноде vLLM автоматически переключается c multiprocessing на Ray для оркестрации.

Комбинация с TP. Итоговое число GPU = tp × pp; например, tp = 8 (8 GPU на узле) и pp = 2 (2 узла) задействуют 16 GPU и позволяют поместить 70-миллиардную модель, не увеличивая задержку в 8 раз
vLLM.

### 6.3 Сетевые настройки и NCCL
TP/PP обмениваются тензорами через NCCL, неверный выбор интерфейса приводит к тайм-аутам и «просадкам» пропускной способности.

- `NCCL_SOCKET_IFNAME=eth0` — явно выбирает NIC, исключая docker0 и прочие виртуальные интерфейсы;
- `NCCL_P2P_DISABLE=1` — отключает peer-to-peer оптимизацию, если узел без NVLink и работает только через PCIe, что ускоряет инициализацию TP;
- `NCCL_DEBUG=warn` — выводит предупреждения-«зависания» барьеров, полезно при первых тестах multi-node.

## 7. Пример структуры Docker Compose
Тут — та же концепция, что и в изначальном примере, но здесь можно поменять `replicas: 1` на большее число (например, `replicas: 2`) в секции `deploy` для `vllm-api`. Nginx будет маршрутизировать запросы.

```yaml
services:
  vllm-api:
    build: ./vllm
    container_name: vllm-api
    environment:
      - NVIDIA_VISIBLE_DEVICES=all
    deploy:
      replicas: 1  # Можно увеличить, если нужно масштабировать, тогда будут создаваться с разными именами
      resources:
        reservations:
          devices:
            - capabilities: [gpu]
    ports:
      - "8000"
      - "9464"
    networks:
      - llm-net

  prometheus:
    image: prom/prometheus:v2.52.0
    container_name: prometheus
    volumes:
      - ./prometheus/prometheus.yml:/etc/prometheus/prometheus.yml:ro
    ports:
      - "9090:9090"
    networks:
      - llm-net

  grafana:
    image: grafana/grafana-oss:11.0.0
    container_name: grafana
    environment:
      - GF_SECURITY_ADMIN_PASSWORD=admin
    volumes:
      - ./grafana/provisioning:/etc/grafana/provisioning:ro
    ports:
      - "3000:3000"
    depends_on:
      - prometheus
    networks:
      - llm-net

  nginx:
    image: nginx:alpine
    container_name: nginx
    volumes:
      - ./nginx/nginx.conf:/etc/nginx/nginx.conf:ro
    ports:
      - "80:80"
    depends_on:
      - vllm-api
    networks:
      - llm-net

networks:
  llm-net:
    driver: bridge
```

Пример `nginx.conf` может содержать:

```nginx
events {
  worker_connections  1024;
}

http {
  upstream vllm_cluster {
    # Несколько экземпляров (реплик) vLLM.
    server vllm-api:8000;
    # Если replicas > 1, дополнительно:
    # server vllm-api-2:8000;
    # server vllm-api-3:8000;
  }

  server {
    listen 80;
    location / {
      proxy_pass http://vllm_cluster;
    }
  }
}
```


## 8. Пример Dockerfile под vLLM (server)
Ниже — Dockerfile, где мы устанавливаем все необходимые зависимости, клонируем `flashinfer`, `vllm` и настраиваем модель `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B`.

```dockerfile
FROM nvcr.io/nvidia/pytorch:25.03-py3

ENV MAX_JOBS=16 \
    NVCC_THREADS=4 \
    FLASHINFER_ENABLE_AOT=0 \
    USE_CUDA=1 \
    CUDA_HOME=/usr/local/cuda \
    TORCH_CUDA_ARCH_LIST='12.0+PTX' \
    CCACHE_DIR=/root/.ccache \
    OTEL_SERVICE_NAME=vllm-api \
    OTEL_METRICS_EXPORTER=prometheus \
    OTEL_TRACES_EXPORTER=none \
    OTEL_EXPORTER_PROMETHEUS_HOST=0.0.0.0 \
    OTEL_EXPORTER_PROMETHEUS_PORT=9464

RUN apt-get update && apt-get install -y --no-install-recommends \
        kmod git cmake ccache python3-pip python3-dev \
    && apt-get clean && rm -rf /var/lib/apt/lists/*

RUN pip3 install --upgrade pip
RUN pip3 install bitsandbytes \
    opentelemetry-distro \
    opentelemetry-exporter-prometheus \
    opentelemetry-instrumentation-fastapi \
    opentelemetry-instrumentation-asgi

RUN git clone --recursive https://github.com/flashinfer-ai/flashinfer.git /workspace/flashinfer
WORKDIR /workspace/flashinfer
RUN pip3 install -e . -v

RUN git clone https://github.com/vllm-project/vllm.git /workspace/vllm
WORKDIR /workspace/vllm
RUN python3 use_existing_torch.py
RUN pip3 install --no-cache-dir -r requirements/build.txt
RUN pip3 install --no-cache-dir setuptools_scm
RUN python3 setup.py develop

RUN pip3 install --no-cache-dir transformers accelerate huggingface_hub

# Скрипт запуска.
COPY scripts/run.sh /workspace/run.sh
EXPOSE 8000 9464
CMD [ "/workspace/run.sh" ]
```


## 9. Скрипт запуска (run.sh)
```bash
#!/usr/bin/env bash
set -e

# Используем OpenTelemetry для экспорта метрик в Prometheus.
exec opentelemetry-instrument \
  --metrics_exporter prometheus \
  --service_name "${OTEL_SERVICE_NAME:-vllm-api}" \
  vllm serve deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B \
        --host 0.0.0.0 \
        --port 8000 \
        --trust-remote-code \
        --dtype bfloat16 \
        --quantization bitsandbytes
```

Обратите внимание, что здесь мы используем **server**-версию `vllm serve`, а не локальный Python-код. Модель `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B` подгружается из репозитория Hugging Face.


## 10. Мониторинг с OpenTelemetry и Prometheus
- **OpenTelemetry** собирает метрики по работе сервиса (время отклика, количество запросов, использование памяти и т. д.).
- **Prometheus** «scrape»-ит эти метрики (порт 9464) и сохраняет их.
- **Grafana** отображает метрики в удобном виде (графики, дашборды).


## 11. Пример (демонстрационный) инференса напрямую (локально)
Хотя мы используем **server-версию**, иногда полезно показать небольшой пример **локального** вызова (не в проде!). Просто для демонстрации принципа.

```python
# Если бы мы запускали локально:
# !pip install vllm
from vllm import LLM, SamplingParams
llm = LLM("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
prompt = "Привет! Расскажи интересный факт о космосе."
sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.9,
    max_tokens=64
)
response = llm.generate([prompt], sampling_params)
print(response[0].outputs[0].text)
```

Однако в вашем случае всё завернуто в контейнер и API, поэтому прямого `llm.generate` нет — вы дергаете REST или gRPC.


## 12. Итог
1. **Запуск**: `docker-compose up -d --build`.
2. **Сервис vLLM** поднимается на порту 8000 (для метрик 9464).
3. **Nginx** слушает 80-й порт и маршрутизирует запросы к `vllm-api`.
4. **Prometheus** (порт 9090) собирает метрики.
5. **Grafana** (порт 3000) показывает дашборды.
6. **Масштабирование**: если нужно обслуживать больше клиентов — увеличивайте `replicas` у `vllm-api` и добавляйте записи `server` в `nginx.conf` (или используйте динамический сервис-дискавери, если это Kubernetes). Так же мы рассмотрели **Tensor Parralelism** и **Pipeline Parralelism**.

Таким образом, мы показали базовую архитектуру **server-версии** vLLM, пояснили, как **распределять нагрузку** между несколькими инстансами, а также коротко описали основные механизмы оптимизаций для LLM-инференса.
